#### <font color=red, size=3>一、任务：</font>
#### 基于RoBERTa训练一个MLM模型

#### <font color=red, size=3>二、流程：</font>
#### 第1步：基于一个txt语料训练一个tokenizer
#### 第2步：定义roberta的超参数
#### 第3步：基于第1步训练的tokenizer，创建一个roberta tokenizer
#### 第4步：基于第2步定义的config，创建一个roberta MLM模型
#### 第5步：模型的总参数量计算
#### 第6步：创建训练数据对象
#### 第7步：定义 训练模型的超参数、并创建模型对象¶
#### 第8步：模型训练、保存¶
#### 第9步：模型预测（示例演示）

#### <font color=red, size=3>三、声明：本案例参考如下链接：</font>

① https://huggingface.co/blog/how-to-train

② https://huggingface.co/docs/transformers/model_doc/roberta

③ https://huggingface.co/docs/transformers/main_classes/trainer

④ https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

⑤ https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForLanguageModeling

⑥ https://discuss.huggingface.co/t/preparing-a-nlp-dataset-for-mlm/3652

In [1]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import os
import json

import torch

from transformers import RobertaConfig, RobertaTokenizer, RobertaModel, RobertaForMaskedLM
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, pipeline

In [2]:
torch.cuda.is_available()

True

### 第1步：基于一个txt语料训练一个tokenizer

In [3]:
# 查找当前目录下的txt文件

texts = [str(x) for x in Path(".").glob("**/*.txt")]

In [4]:
texts

['shakespeare.txt']

In [5]:
# 初始化一个 tokenizer

tokenizer = ByteLevelBPETokenizer()

print(tokenizer)

Tokenizer(vocabulary_size=0, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)


In [6]:
%%time
# 自定义 tokinzer 训练

tokenizer.train(files=texts, # 训练语料
                vocab_size=52000, # 词汇量
                min_frequency=2, # 单词出现频次，如果低于该阈值，则忽略
                special_tokens=['<s>','<pad>','</s>','<unk>','<mask>']) # 特殊词
'''
<s> : 开始标记
</s> : 结束标记
<unk> : 未知标记
<mask> : 遮盖标记
<pad> : 填充标记
'''




CPU times: user 6.87 s, sys: 1.69 s, total: 8.57 s
Wall time: 760 ms


'\n<s> : 开始标记\n</s> : 结束标记\n<unk> : 未知标记\n<mask> : 遮盖标记\n<pad> : 填充标记\n'

In [7]:
# 保存tokenizer到本地

token_dir = './robertaModel' # 存放tokenizer的文件夹路径

if not os.path.exists(token_dir):
    os.makedirs(token_dir) # 创建文件夹

tokenizer.save_model("robertaModel")

['robertaModel/vocab.json', 'robertaModel/merges.txt']

In [8]:
# 分词示例

tokenizer.encode("the rabbit is jumping.").tokens

['the', 'Ġrabbit', 'Ġis', 'Ġjump', 'ing', '.']

In [9]:
result = tokenizer.encode("the rabbit is jumping.")

In [10]:
result

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [11]:
result.ids # 每一个token对应的id

[2038, 18903, 341, 9637, 309, 18]

In [12]:
result.type_ids # 全0表示一句话

[0, 0, 0, 0, 0, 0]

In [13]:
result.tokens # 每一个token

['the', 'Ġrabbit', 'Ġis', 'Ġjump', 'ing', '.']

In [14]:
result.attention_mask # 全1表示没有mask操作

[1, 1, 1, 1, 1, 1]

In [15]:
result.special_tokens_mask # 全0表示没有special tokens

[0, 0, 0, 0, 0, 0]

In [16]:
# 定义后处理的方式，即在每句话的前后加上：<s> 和 </s>

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>"))
)

In [17]:
# 最长的语句不超过512个token

tokenizer.enable_truncation(max_length=512)

In [18]:
tokenizer.save('robertaModel/config.json')

### 第2步：定义roberta的超参数

In [19]:
# 定义模型的配置

config = RobertaConfig(
    vocab_size = 52000, # 词汇表大小
    max_position_embeddings = 514, # position的大小
    num_attention_heads = 12, # attention 头
    num_hidden_layers = 6, # 6层
    type_vocab_size = 1 # 指代 token_type_ids 的类别
)

In [20]:
config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

### 第3步：基于第1步训练的tokenizer，创建一个roberta tokenizer

In [21]:
# 创建 Roberta tokenizer

roberta_tokenizer = RobertaTokenizer.from_pretrained('./robertaModel/', max_length=512)

### 第4步：基于第2步定义的config，创建一个roberta MLM模型

In [22]:
# 初始化 model

roberta_model = RobertaForMaskedLM(config=config)

In [23]:
roberta_model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [24]:
print(roberta_model.num_parameters()) # 总的模型参数量

83504416


In [25]:
list(roberta_model.parameters())

[Parameter containing:
 tensor([[ 0.0140,  0.0067, -0.0328,  ..., -0.0048, -0.0243,  0.0014],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0293,  0.0067,  0.0214,  ...,  0.0125,  0.0045, -0.0094],
         ...,
         [ 0.0303, -0.0091, -0.0189,  ...,  0.0088, -0.0343,  0.0109],
         [-0.0427,  0.0036,  0.0525,  ...,  0.0120, -0.0099, -0.0154],
         [-0.0189,  0.0550,  0.0259,  ..., -0.0488, -0.0040, -0.0047]],
        requires_grad=True),
 Parameter containing:
 tensor([[ 0.0033, -0.0285, -0.0084,  ..., -0.0141, -0.0035, -0.0299],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0307,  0.0139,  0.0279,  ...,  0.0218,  0.0262, -0.0344],
         ...,
         [ 0.0032, -0.0100,  0.0362,  ...,  0.0017, -0.0079,  0.0515],
         [-0.0405,  0.0190,  0.0184,  ..., -0.0016, -0.0142,  0.0211],
         [-0.0249, -0.0165, -0.0144,  ...,  0.0075,  0.0147, -0.0099]],
        requires_grad=True),
 Parameter con

In [26]:
params = list(roberta_model.parameters())

num_params = len(params)

num_params

106

In [27]:
for p in range(0, num_params):
    print(params[p])
    if p==0:
        break

Parameter containing:
tensor([[ 0.0140,  0.0067, -0.0328,  ..., -0.0048, -0.0243,  0.0014],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0293,  0.0067,  0.0214,  ...,  0.0125,  0.0045, -0.0094],
        ...,
        [ 0.0303, -0.0091, -0.0189,  ...,  0.0088, -0.0343,  0.0109],
        [-0.0427,  0.0036,  0.0525,  ...,  0.0120, -0.0099, -0.0154],
        [-0.0189,  0.0550,  0.0259,  ..., -0.0488, -0.0040, -0.0047]],
       requires_grad=True)


### 第5步：模型的总参数量计算

In [28]:
# 模型参数计算，即 83504416 怎么算出来的？

sum = 0

for i in range(p, num_params):
    flag = True
    try:
        dim = len(params[i][0]) # 判断是否有第2维，如果有，则计算第2维的大小
    except:
        dim = 1 # 只有1维
        flag = False
    
    first_dim = len(params[i]) # 第1维大小
    second_dim = dim # 第2维大小
    all_size = first_dim * second_dim # 总的参数量
    
    # 累计每一层的参数量
    sum += all_size
    
    if flag == True:
        # 该层有2个维度
        print(f'第 {i} 层，第一维度：{first_dim}，第二维度：{second_dim}，该层参数量：{all_size}')
    else:
        # 该层有1个维度
        print(f'第 {i} 层，维度：{first_dim}, 该层参数量：{all_size}')
        
print("\n\n")
print(f'总的参数量{sum}')

第 0 层，第一维度：52000，第二维度：768，该层参数量：39936000
第 1 层，第一维度：514，第二维度：768，该层参数量：394752
第 2 层，第一维度：1，第二维度：768，该层参数量：768
第 3 层，维度：768, 该层参数量：768
第 4 层，维度：768, 该层参数量：768
第 5 层，第一维度：768，第二维度：768，该层参数量：589824
第 6 层，维度：768, 该层参数量：768
第 7 层，第一维度：768，第二维度：768，该层参数量：589824
第 8 层，维度：768, 该层参数量：768
第 9 层，第一维度：768，第二维度：768，该层参数量：589824
第 10 层，维度：768, 该层参数量：768
第 11 层，第一维度：768，第二维度：768，该层参数量：589824
第 12 层，维度：768, 该层参数量：768
第 13 层，维度：768, 该层参数量：768
第 14 层，维度：768, 该层参数量：768
第 15 层，第一维度：3072，第二维度：768，该层参数量：2359296
第 16 层，维度：3072, 该层参数量：3072
第 17 层，第一维度：768，第二维度：3072，该层参数量：2359296
第 18 层，维度：768, 该层参数量：768
第 19 层，维度：768, 该层参数量：768
第 20 层，维度：768, 该层参数量：768
第 21 层，第一维度：768，第二维度：768，该层参数量：589824
第 22 层，维度：768, 该层参数量：768
第 23 层，第一维度：768，第二维度：768，该层参数量：589824
第 24 层，维度：768, 该层参数量：768
第 25 层，第一维度：768，第二维度：768，该层参数量：589824
第 26 层，维度：768, 该层参数量：768
第 27 层，第一维度：768，第二维度：768，该层参数量：589824
第 28 层，维度：768, 该层参数量：768
第 29 层，维度：768, 该层参数量：768
第 30 层，维度：768, 该层参数量：768
第 31 层，第一维度：3072，第二维度：768，该层参数量：2359296
第 32 层，维度：3072, 该层参数量

### 第6步：创建训练数据对象

In [29]:
# 创建 dataset, data collector

dataset = LineByLineTextDataset(tokenizer=roberta_tokenizer, # 分词器
                                file_path='shakespeare.txt', # 文本数据
                                block_size=128) # 每批读取128行

/home/tgl/torch_env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [30]:
data_collector = DataCollatorForLanguageModeling(tokenizer=roberta_tokenizer, # 分词器
                                                 mlm=True, # mlm模型
                                                 mlm_probability=0.15) # 15%的word进行mask

# https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForLanguageModeling

### 第7步：定义 训练模型的超参数、并创建模型对象

In [31]:
# 初始化 trainer

# ① 参数定义

trainArgs = TrainingArguments(
    output_dir='./roberta_output/', # 输出路径
    overwrite_output_dir=True, # 可以覆盖之前的输出
    do_train=True, # 训练
    num_train_epochs=1 # 默认值
)

# https://discuss.huggingface.co/t/preparing-a-nlp-dataset-for-mlm/3652

In [32]:
# ② trainer 定义

trainer = Trainer(
    model=roberta_model, # 模型对象
    args=trainArgs, # 训练参数
    data_collator=data_collector, # collector
    train_dataset=dataset # 数据集
)

### 第8步：模型训练、保存

In [33]:
# 预训练模型

trainer.train()

***** Running training *****
  Num examples = 114840
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14355


Step,Training Loss
500,7.315600
1000,6.427400
1500,6.184800
2000,6.020800
2500,5.851600
3000,5.899100
3500,5.765700
4000,5.621900
4500,5.589400
5000,5.516800


Saving model checkpoint to ./roberta_output/checkpoint-500
Configuration saved in ./roberta_output/checkpoint-500/config.json
Model weights saved in ./roberta_output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./roberta_output/checkpoint-1000
Configuration saved in ./roberta_output/checkpoint-1000/config.json
Model weights saved in ./roberta_output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./roberta_output/checkpoint-1500
Configuration saved in ./roberta_output/checkpoint-1500/config.json
Model weights saved in ./roberta_output/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./roberta_output/checkpoint-2000
Configuration saved in ./roberta_output/checkpoint-2000/config.json
Model weights saved in ./roberta_output/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./roberta_output/checkpoint-2500
Configuration saved in ./roberta_output/checkpoint-2500/config.json
Model weights saved in ./roberta_output/checkpoint-2500/pytorch_mod

TrainOutput(global_step=14355, training_loss=5.400965711805555, metrics={'train_runtime': 402.9156, 'train_samples_per_second': 285.022, 'train_steps_per_second': 35.628, 'total_flos': 542036745429504.0, 'train_loss': 5.400965711805555, 'epoch': 1.0})

In [34]:
# 保存模型

trainer.save_model("./robertaModel/")

Saving model checkpoint to ./robertaModel/
Configuration saved in ./robertaModel/config.json
Model weights saved in ./robertaModel/pytorch_model.bin


### 第9步：模型预测（示例演示）

In [35]:
# 模型预测示例

predict_mask = pipeline('fill-mask',
                        model='./robertaModel/',
                        tokenizer='./robertaModel/')

loading configuration file ./robertaModel/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./robertaModel/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dro

In [36]:
predict_mask("Great man make great <mask>.")

[{'sequence': 'Great man make great man.',
  'score': 0.0066635883413255215,
  'token': 463,
  'token_str': ' man'},
 {'sequence': 'Great man make great hand.',
  'score': 0.004996541887521744,
  'token': 646,
  'token_str': ' hand'},
 {'sequence': 'Great man make great me.',
  'score': 0.004365968983620405,
  'token': 331,
  'token_str': ' me'},
 {'sequence': 'Great man make great out.',
  'score': 0.004032505676150322,
  'token': 612,
  'token_str': ' out'},
 {'sequence': 'Great man make great head.',
  'score': 0.0037917550653219223,
  'token': 920,
  'token_str': ' head'}]

In [39]:
'''
But flowers distilled though they with winter meet,
    Leese but their show, their substance still lives sweet.
'''
predict_mask("But flowers distilled though they with <mask> meet,Leese but their show, \
              their substance still lives sweet.")

[{'sequence': 'But flowers distilled though they with the meet,Leese but their show,               their substance still lives sweet.',
  'score': 0.2046729177236557,
  'token': 275,
  'token_str': ' the'},
 {'sequence': 'But flowers distilled though they with a meet,Leese but their show,               their substance still lives sweet.',
  'score': 0.10849585384130478,
  'token': 265,
  'token_str': ' a'},
 {'sequence': 'But flowers distilled though they with my meet,Leese but their show,               their substance still lives sweet.',
  'score': 0.06749602407217026,
  'token': 322,
  'token_str': ' my'},
 {'sequence': 'But flowers distilled though they with his meet,Leese but their show,               their substance still lives sweet.',
  'score': 0.039168041199445724,
  'token': 370,
  'token_str': ' his'},
 {'sequence': 'But flowers distilled though they with your meet,Leese but their show,               their substance still lives sweet.',
  'score': 0.03280680626630783,
  'to